# NVPC Chatbot
This is a practice chatbot. All databases used in this bot are either public or fictional.

#### Functions:
- provides instant answers to common questions, based on giving.sg FAQ
- suggests campaigns that are most similar to user's interest

### Setting up Environment

In [1]:
import os
import json
import re
import numpy as np
from langchain.llms import OpenAI
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import gradio as gr
from rapidfuzz import process, fuzz

In [2]:
os.environ["OPENAI_API_KEY"] = "voc-183321194312667737058046757cd49b1d8b6.91865541"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

model_name = "gpt-3.5-turbo"
temperature = 0.0
llm = OpenAI(model_name=model_name, temperature=temperature, max_tokens=2000)

# Initialize an embedding model for semantic search
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

C:\Users\Olivi\anaconda3\envs\Jupyter_env\Lib\site-packages\langchain\llms\openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\Olivi\anaconda3\envs\Jupyter_env\Lib\site-packages\langchain\llms\openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### FAQ Functionality

In [3]:
faqs = [
    {
        "question": "What is giving.sg?",
        "answer": "giving.sg is a platform that connects individuals with social campaigns and charitable causes in Singapore."
    },
    {
        "question": "How do I donate?",
        "answer": "You can donate by visiting the specific campaign page on giving.sg and following the donation instructions provided."
    },
    {
        "question": "How do I start a campaign?",
        "answer": "To start a campaign, register on giving.sg, submit your campaign details, and wait for approval from the giving.sg team."
    },
    {
        "question": "Is my donation tax-deductible?",
        "answer": "Yes, donations made via giving.sg may be eligible for tax deductions, depending on the charity's registration and local tax regulations."
    },
    {
        "question": "How can I volunteer?",
        "answer": "If you wish to volunteer, check out the volunteer opportunities listed on giving.sg or contact the charity directly."
    },
    {
        "question": "What types of campaigns are featured?",
        "answer": "giving.sg features a wide range of campaigns including education, healthcare, community development, environmental causes, and more."
    },
    {
        "question": "How do I contact support?",
        "answer": "For support, please visit the 'Contact Us' section on giving.sg where you can find our email and hotline details."
    },
    {
        "question": "What is the mission of giving.sg?",
        "answer": "The mission of giving.sg is to create a more connected and compassionate community by bridging the gap between those who want to help and those in need."
    }
]

In [4]:
# Pre-compute embeddings for FAQ questions.
faq_questions = [faq["question"] for faq in faqs]
faq_embeddings = embedding_model.encode(faq_questions)

In [5]:
def answer_faq(user_question, threshold=0.7):
    """
    Answer a user's FAQ question by finding the closest match in the FAQ dataset.
    If no close match is found, fallback to the LLM to generate a response.
    """
    user_embedding = embedding_model.encode([user_question])
    similarities = cosine_similarity(user_embedding, faq_embeddings)[0]
    best_idx = np.argmax(similarities)
    if similarities[best_idx] >= threshold:
        return faqs[best_idx]["answer"]
    else:
        prompt = f"I have an FAQ sheet for giving.sg. When a user asks: '{user_question}', provide a helpful answer using that FAQ as reference."
        return llm(prompt)

### Campaign Recommendation Functionality

In [6]:
# Defining a sample campaigns dataset (11 listings)
campaigns = [
    {
        "title": "Education for All",
        "target_demographic": "Underprivileged children and youth",
        "charity_name": "EduHope",
        "about_charity": "EduHope provides free tutoring, scholarships, and educational resources to children in need across Singapore.",
        "how_to_support": "Donate funds, volunteer as a tutor, or provide school supplies."
    },
    {
        "title": "Healthcare for All",
        "target_demographic": "Low-income families",
        "charity_name": "HealthFirst",
        "about_charity": "HealthFirst delivers essential medical services and subsidized healthcare to underserved communities.",
        "how_to_support": "Donate money or volunteer at community clinics."
    },
    {
        "title": "Elderly Empowerment",
        "target_demographic": "Senior citizens",
        "charity_name": "GoldenYears",
        "about_charity": "GoldenYears offers assistance programs, social events, and community support for the elderly.",
        "how_to_support": "Donate or volunteer time to assist seniors."
    },
    {
        "title": "Youth Empowerment",
        "target_demographic": "Young adults and students",
        "charity_name": "FutureLeaders",
        "about_charity": "FutureLeaders mentors and trains youth to develop leadership and entrepreneurial skills.",
        "how_to_support": "Donate, mentor, or sponsor educational workshops."
    },
    {
        "title": "Community Green Spaces",
        "target_demographic": "Urban communities",
        "charity_name": "GreenCity",
        "about_charity": "GreenCity creates and maintains public parks and community gardens to promote environmental wellness.",
        "how_to_support": "Donate funds or volunteer for garden projects."
    },
    {
        "title": "Animal Welfare",
        "target_demographic": "Animal lovers and pet owners",
        "charity_name": "Paws & Claws",
        "about_charity": "Paws & Claws rescues abandoned animals and provides veterinary care and shelter.",
        "how_to_support": "Donate, foster animals, or volunteer at shelters."
    },
    {
        "title": "Women Empowerment",
        "target_demographic": "Women and girls",
        "charity_name": "SheRise",
        "about_charity": "SheRise provides career training, mentorship, and support programs to empower women.",
        "how_to_support": "Donate funds or volunteer as a mentor."
    },
    {
        "title": "Disaster Relief",
        "target_demographic": "Communities affected by disasters",
        "charity_name": "ReliefNow",
        "about_charity": "ReliefNow provides immediate aid and long-term recovery support for disaster-stricken areas.",
        "how_to_support": "Donate funds, supplies, or volunteer for relief efforts."
    },
    {
        "title": "Mental Health Awareness",
        "target_demographic": "General public",
        "charity_name": "MindCare",
        "about_charity": "MindCare raises awareness, offers counseling, and supports mental health initiatives.",
        "how_to_support": "Donate or participate in mental health awareness events."
    },
    {
        "title": "Environmental Sustainability",
        "target_demographic": "Environment advocates",
        "charity_name": "EcoFuture",
        "about_charity": "EcoFuture works on sustainability projects and environmental conservation efforts.",
        "how_to_support": "Donate funds or volunteer in sustainability programs."
    },
    {
        "title": "Food for the Needy",
        "target_demographic": "Low-income communities",
        "charity_name": "FoodShare",
        "about_charity": "FoodShare collects surplus food and distributes it to families in need.",
        "how_to_support": "Donate funds or contribute food items."
    }
]

### Query expansion with keywords and fuzzy matching

In [7]:
def get_campaign_keywords(campaigns):
    """
    Extract unique keywords from all fields of the campaign database.
    """
    all_text = " ".join(
        campaign["title"] + " " + campaign["target_demographic"] + " " +
        campaign["charity_name"] + " " + campaign["about_charity"] + " " +
        campaign["how_to_support"] for campaign in campaigns
    )
    words = re.findall(r'\w+', all_text.lower())
    # Remove common stopwords and short words
    stopwords = {"the", "and", "a", "of", "to", "in", "for", "with", "on", "at", "by", "is", "are", "as"}
    filtered_words = [word for word in words if word not in stopwords and len(word) > 2]
    return list(set(filtered_words))

# Get the list of keywords from the campaign database.
campaign_keywords = get_campaign_keywords(campaigns)

In [8]:
def expand_query(query, keywords, threshold=80):
    """
    Expand the user query by comparing each word in the query to the list of campaign keywords.
    If a keyword is a close fuzzy match to a query word (above the threshold), add it to the query.
    """
    query_words = re.findall(r'\w+', query.lower())
    expanded = set(query_words)
    for qword in query_words:
        # Find keywords that have a high fuzzy match score with qword.
        matches = process.extract(qword, keywords, scorer=fuzz.ratio, score_cutoff=threshold)
        for match, score, _ in matches:
            expanded.add(match)
    return " ".join(expanded)

### Personalisation

In [15]:
def recommend_campaigns(user_interests, threshold=0.3):
    """
    Recommend campaigns based on the user's interests.
    The user query is expanded using all keywords from the campaign database via fuzzy matching.
    """
    expanded_query = expand_query(user_interests, campaign_keywords)
    query_embedding = embedding_model.encode([expanded_query])
    recommended = []
    for campaign in campaigns:
        # Combine several fields for a richer representation.
        combined_text = f"{campaign['title']} {campaign['target_demographic']} {campaign['about_charity']}"
        combined_embedding = embedding_model.encode([combined_text])
        sim = cosine_similarity(query_embedding, combined_embedding)[0][0]
        if sim >= threshold:
            recommended.append((campaign, sim))
    # Optionally, sort by similarity.
    recommended.sort(key=lambda x: x[1], reverse=True)
    return [r[0] for r in recommended]


In [16]:
def personalized_campaign_summary(campaign, user_interests):
    """
    Use the LLM to generate a personalized paragraph that explains why
    the campaign matches the user's interests, including detailed campaign information.
    """
    prompt = f"""
You are a helpful assistant tasked with explaining how a charitable campaign matches a user's interests. Below are the details:

User Interests: {user_interests}

Campaign Details:
Title: {campaign['title']}
Target Demographic: {campaign['target_demographic']}
Charity Name: {campaign['charity_name']}
About the Charity: {campaign['about_charity']}
How to Support: {campaign['how_to_support']}

Write a detailed, engaging paragraph that explains how this campaign is a great match for the user's interests, and include all the relevant details.
"""
    explanation = llm(prompt)
    return explanation.strip()

In [17]:
def campaign_details(user_interests):
    """
    Returns a personalized output with detailed campaign information
    for campaigns matching the user's interests.
    """
    recommended = recommend_campaigns(user_interests)
    if not recommended:
        return "No campaigns match your interests. Please try a different description."
    
    output_str = ""
    for campaign in recommended:
        personalized_text = personalized_campaign_summary(campaign, user_interests)
        output_str += personalized_text + "\n\n" + "-"*50 + "\n\n"
    return output_str

### GRADIO INTERFACE FUNCTIONS

In [18]:
def faq_bot(user_question):
    """Wrapper for answering FAQ questions."""
    return answer_faq(user_question)

def campaign_bot(user_interests):
    """Wrapper for providing detailed campaign recommendations."""
    return campaign_details(user_interests)

In [19]:
# Create two Gradio interfaces for the FAQ and Campaign recommendation functionalities.
faq_interface = gr.Interface(
    fn=faq_bot,
    inputs=gr.Textbox(lines=2, placeholder="Enter your FAQ question here...", label="FAQ Question"),
    outputs=gr.Textbox(label="Answer"),
    title="giving.sg FAQ Chatbot",
    description="Ask any questions related to giving.sg. This uses the FAQ dataset from the giving.sg FAQ page."
)

campaign_interface = gr.Interface(
    fn=campaign_bot,
    inputs=gr.Textbox(lines=2, placeholder="Describe your campaign interests here...", label="Your Interests"),
    outputs=gr.Textbox(label="Recommended Campaigns"),
    title="giving.sg Campaign Recommender",
    description="Get detailed campaign recommendations based on your interests."
)

# Combine the two interfaces into a tabbed layout.
tabbed_interface = gr.TabbedInterface([faq_interface, campaign_interface], tab_names=["FAQ", "Campaigns"])

In [20]:
# Launch the Gradio demo.
tabbed_interface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
